# Example 2: Coupled Transmon Qubits

In [1]:
%load_ext autoreload
%autoreload 2
import time

In [2]:
import json
import design as d
import names as n

with open("design_variables.json") as in_file:
    initial_design_variables = json.load(in_file)

from qdesignoptimizer.utils.chip_generation import create_chip_base, ChipType
from qdesignoptimizer.utils.utils import close_ansys


## Design assembly

In [3]:
CHIP_NAME = "multi_transmon_chip"
OPEN_GUI = True
chip_type = ChipType(size_x="10mm", size_y="10mm", size_z="-300um")
design, gui = create_chip_base(
    chip_name=CHIP_NAME, chip_type=chip_type, open_gui=OPEN_GUI
)

n.add_design_variables_to_design(design, initial_design_variables)

In [4]:
def render_qiskit_metal_design(design, gui):
    d.add_transmon_plus_resonator(design, group=n.GROUP_1)
    d.add_transmon_plus_resonator(design, group=n.GROUP_2)

    d.add_coupler(design)

    d.add_route_interconnects(design)

    d.add_launch_pads(design)

    d.add_chargeline(design, group=n.GROUP_1)
    d.add_chargeline(design, group=n.GROUP_2)

    gui.rebuild()
    gui.autoscale()


render_qiskit_metal_design(design, gui)

## MiniStudies

In [5]:
import mini_studies as ms
import optimization_targets as ot
import parameter_targets as pt
import plot_settings as ps

from qdesignoptimizer.design_analysis import DesignAnalysis, DesignAnalysisState

### Single resonator-qubit system
Useful when first tuning up subsystems

In [6]:
MINI_STUDY_GROUP = n.GROUP_1
MINI_STUDY = ms.get_mini_study_qb_res(group=MINI_STUDY_GROUP)
RENDER_QISKIT_METAL = lambda design: render_qiskit_metal_design(design, gui)

opt_targets = ot.get_opt_targets_2qubits_resonator_coupler(
    groups=[MINI_STUDY_GROUP],
    opt_target_qubit_freq=True,
    opt_target_qubit_anharm=True,
    opt_target_resonator_freq=True,
    opt_target_resonator_kappa=False,
    opt_target_resonator_qubit_chi=True,
)

In [ ]:
design_analysis_state = DesignAnalysisState(
    design, RENDER_QISKIT_METAL, pt.PARAM_TARGETS
)
design_analysis = DesignAnalysis(
    design_analysis_state,
    mini_study=MINI_STUDY,
    opt_targets=opt_targets,
    save_path='out/' + CHIP_NAME + "_" + time.strftime("%Y%m%d-%H%M%S"),
    update_design_variables=False,
    plot_settings=ps.PLOT_SETTINGS,
)

group_runs = 2  # 10
group_passes = 2  # 6
delta_f = 0.001
for i in range(group_runs):
    design_analysis.update_nbr_passes(group_passes)
    design_analysis.update_delta_f(delta_f)
    design_analysis.optimize_target({}, {})
    design_analysis.screenshot(gui=gui, run=i)

[INFO|2025-02-26 15:11:20]: self.eig_solver.sim.setup 
{
    "name": "Resonator_setup",
    "reuse_selected_design": true,
    "reuse_setup": true,
    "min_freq_ghz": 1,
    "n_modes": 1,
    "max_delta_f": 0.5,
    "max_passes": 10,
    "min_passes": 1,
    "min_converged": 1,
    "pct_refinement": 30,
    "basis_order": 1,
    "vars": {
        "Lj": "10 nH",
        "Cj": "0 fF"
    }
}
INFO 03:11PM [connect_project]: Connecting to Ansys Desktop API...
INFO 03:11PM [load_ansys_project]: 	Opened Ansys App
INFO 03:11PM [load_ansys_project]: 	Opened Ansys Desktop v2021.2.0
INFO 03:11PM [load_ansys_project]: 	Opened Ansys Project
	Folder:    C:/Users/lukassp/Documents/Ansoft/
	Project:   Project161
INFO 03:11PM [connect_design]: 	Opened active design
	Design:    get_mini_study_qb_res [Solution type: Eigenmode]
INFO 03:11PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)
INFO 03:11PM [connect]: 	Connected to project "Project161" and design "get_mini_study_qb_res"

pyaedt INFO: using existing logger.
pyaedt INFO: Launching PyAEDT outside AEDT with CPython and PythonNET.
pyaedt INFO: AEDT installation Path C:\Program Files\AnsysEM\AnsysEM21.2\Win64.
pyaedt INFO: Launching AEDT with module PythonNET.
pyaedt INFO: Ansoft.ElectronicsDesktop.2021.2 Started with process ID 15008.
pyaedt INFO: pyaedt v0.6.46
pyaedt INFO: Python version 3.10.15 | packaged by Anaconda, Inc. | (main, Oct  3 2024, 07:22:19) [MSC v.1929 64 bit (AMD64)]
pyaedt INFO: No project is defined. Project Project161 exists and has been read.
pyaedt INFO: Active Design set to get_mini_study_qb_res
pyaedt INFO: Aedt Objects initialized
pyaedt INFO: Variable Manager initialized
pyaedt INFO: Design Loaded
pyaedt INFO: Successfully loaded project materials !
pyaedt INFO: Materials Loaded


[INFO|2025-02-26 15:11:36]: No fine mesh map was found for name_qubit_1
[INFO|2025-02-26 15:11:36]: No fine mesh map was found for name_resonator_1
[INFO|2025-02-26 15:11:36]: No fine mesh map was found for name_tee1
INFO 03:11PM [analyze]: Analyzing setup Setup


Design "get_mini_study_qb_res" info:
	# eigenmodes    2
	# variations    1
Design "get_mini_study_qb_res" info:
	# eigenmodes    2
	# variations    1
Design "get_mini_study_qb_res" info:
	# eigenmodes    2
	# variations    1
Design "get_mini_study_qb_res" info:
	# eigenmodes    2
	# variations    1
Design "get_mini_study_qb_res" info:
	# eigenmodes    2
	# variations    1

        energy_elec_all       = 6.37386658521156e-23
        energy_elec_substrate = 5.89822760899604e-23
        EPR of substrate = 92.5%

        energy_mag    = 6.3645203878594e-23
        energy_mag % of energy_elec_all  = 99.9%
        

Variation 0  [1/1]

  Mode 0 at 4.74 GHz   [1/2]
    Calculating ℰ_magnetic,ℰ_electric
       (ℰ_E-ℰ_H)/ℰ_E       ℰ_E       ℰ_H
               98.6%  4.312e-24 5.839e-26

    Calculating junction energy participation ration (EPR)
	method=`line_voltage`. First estimates:
	junction        EPR p_0j   sign s_0j    (p_capacitive)
		Energy fraction (Lj over Lj&Cj)= 98.24%
	jj_name_qub

WARNING 03:11PM [__init__]: <p>Error: <class 'IndexError'></p>


...

Design "get_mini_study_qb_res" info:
	# eigenmodes    2
	# variations    1
Design "get_mini_study_qb_res" info:
	# eigenmodes    2
	# variations    1
Design "get_mini_study_qb_res" info:
	# eigenmodes    2
	# variations    1
Design "get_mini_study_qb_res" info:
	# eigenmodes    2
	# variations    1
Design "get_mini_study_qb_res" info:
	# eigenmodes    2
	# variations    1

        energy_elec_all       = 7.75772199315758e-23
        energy_elec_substrate = 7.15759011763071e-23
        EPR of substrate = 92.3%

        energy_mag    = 7.74700167040488e-23
        energy_mag % of energy_elec_all  = 99.9%
        

Variation 0  [1/1]

  Mode 0 at 3.75 GHz   [1/2]
    Calculating ℰ_magnetic,ℰ_electric
       (ℰ_E-ℰ_H)/ℰ_E       ℰ_E       ℰ_H
               99.0%  1.914e-24 1.951e-26

    Calculating junction energy participation ration (EPR)
	method=`line_voltage`. First estimates:
	junction        EPR p_0j   sign s_0j    (p_capacitive)
		Energy fraction (Lj over Lj&Cj)= 98.43%
	jj_name_qu

WARNING 03:12PM [__init__]: <p>Error: <class 'IndexError'></p>



ANALYSIS DONE. Data saved to:

C:\data-pyEPR\Project161\get_mini_study_qb_res\2025-02-26 15-12-26.npz


	 Differences in variations:



 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 
Variation 0

Starting the diagonalization
Finished the diagonalization
Pm_norm=
modes
0    1.017503
1    1.004803
dtype: float64

Pm_norm idx =
   jj_name_qubit_1
0             True
1            False
*** P (participation matrix, not normlz.)
   jj_name_qubit_1
0         0.972849
1         0.001424

*** S (sign-bit matrix)
   s_jj_name_qubit_1
0                  1
1                 -1
*** P (participation matrix, normalized.)
      0.99
    0.0014

*** Chi matrix O1 PT (MHz)
    Diag is anharmonicity, off diag is full cross-Kerr.
       151    0.682
     0.682  0.00077

*** Chi matrix ND (MHz) 
       166    0.609
     0.609 0.000611

*** Frequencies O1 PT (MHz)
0    3595.308903
1    5880.119478
dtype: float64

*** Frequencies ND (MHz)
0    3588.571718
1    5880.128245
d

[INFO|2025-02-26 15:12:30]: freq_ND_results
{
    "0": {
        "0": 3588.57171825484,
        "1": 5880.128245272245
    }
}


0 <qdesignoptimizer.sim_plot_progress.OptPltSet object at 0x000002CB8A51AB60> [1, 2] [6818373979.996423, 5880128245.272245]
0 <qdesignoptimizer.sim_plot_progress.OptPltSet object at 0x000002CB8A51AB60> [1, 2] [6818373979.996423, 5880128245.272245]
1 <qdesignoptimizer.sim_plot_progress.OptPltSet object at 0x000002CC0831FFA0> [1, 2] [90620.7943802, 101993.1761386]
2 <qdesignoptimizer.sim_plot_progress.OptPltSet object at 0x000002CC0831C220> [1, 2] [None, None]
0 <qdesignoptimizer.sim_plot_progress.OptPltSet object at 0x000002CC0831C280> [1, 2] [4561152878.0103655, 3588571718.25484]
1 <qdesignoptimizer.sim_plot_progress.OptPltSet object at 0x000002CC0831C2E0> [1, 2] [182829410.9785595, 165552199.28866482]
0 <qdesignoptimizer.sim_plot_progress.OptPltSet object at 0x000002CC0831C340> [1, 2] [609490.586265564, 609358.3860549926]


In [ ]:
design_analysis.overwrite_parameters()

### Two qubit-resonator system

In [ ]:
MINI_STUDY = ms.get_mini_study_2qb_resonator_coupler()
RENDER_QISKIT_METAL = lambda design: render_qiskit_metal_design(design, gui)

opt_targets = ot.get_opt_targets_2qubits_resonator_coupler(
    groups=[n.GROUP_1, n.GROUP_2],
    opt_target_qubit_freq=True,
    opt_target_qubit_anharm=True,
    opt_target_resonator_freq=True,
    opt_target_resonator_kappa=False,
    opt_target_resonator_qubit_chi=True,
    opt_target_coupler_freq=True,
)

In [ ]:
design_analysis_state = DesignAnalysisState(
    design, RENDER_QISKIT_METAL, pt.PARAM_TARGETS
)
design_analysis = DesignAnalysis(
    design_analysis_state,
    mini_study=MINI_STUDY,
    opt_targets=opt_targets,
    save_path=CHIP_NAME + "_" + time.strftime("%Y%m%d-%H%M%S"),
    update_design_variables=False,
    plot_settings=ps.PLOT_SETTINGS,
)

group_runs = 2
group_passes = 3
delta_f = 0.001
for i in range(group_runs):
    design_analysis.update_group_passes(group_passes)
    design_analysis.update_delta_f(delta_f)
    design_analysis.optimize_target({}, {})
    design_analysis.screenshot(gui=gui, run=i)

In [ ]:
design_analysis.overwrite_parameters()

### Purcell decay into charge line

The T1 of the qubit is limited by its decay into the charge line. This ministudy will change the distance of the charge line from the transmon pocket to optimize a T1 limit of 1 ms. This value is computed with simple capacitance matrix simulations and a classical model, assuming that the qubit frequency has already been optimized.

In [ ]:
MINI_STUDY_BRANCH = 1
MINI_STUDY = ms.get_mini_study_qb_charge_line(group=MINI_STUDY_BRANCH)
opt_targets = ot.get_opt_targets_qb_charge_line(group=MINI_STUDY_BRANCH)
RENDER_QISKIT_METAL = lambda design: render_qiskit_metal_design(design, gui)

In [ ]:
# %matplotlib inline
design_analysis_state = DesignAnalysisState(
    design, RENDER_QISKIT_METAL, pt.PARAM_TARGETS
)
design_analysis = DesignAnalysis(
    design_analysis_state,
    mini_study=MINI_STUDY,
    opt_targets=opt_targets,
    save_path=CHIP_NAME + "_" + time.strftime("%Y%m%d-%H%M%S"),
    update_design_variables=False,
    plot_settings=ps.PLOT_SETTINGS_CHARGE_LINE_DECAY,
)

In [ ]:
group_runs = 3
group_passes_cap = 2

for i in range(group_runs):
    design_analysis.update_group_passes_capacitance_ministudies(group_passes_cap)
    design_analysis.optimize_target({}, {})
    design_analysis.screenshot(gui=gui, run=i)

## View Optimization results

In [ ]:
design_analysis.get_cross_kerr_matrix(iteration=-1)

In [ ]:
close_ansys()